## Investigação De Câncer relacionado ao Trabalho - Notificações registradas no Sinan Net - Brasil
## Notificações por CID Câncer
Período: 2019-2023

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/tratado/notificacao_cid_cancer.csv', sep=';')
data

,CID Cancer,Notificações
0,C00 - Neopl malig do labio,3
1,C01 - Neopl malig da base da lingua,8
2,C02 - Neopl malig outr partes e NE da lingua,7
3,C04 - Neopl malig do assoalho da boca,4
4,C05 - Neopl malig do palato,3
...,...,...
84,D46 - Sindr mielodisplasicas,6
85,D48 - Neopl comp inc/desconh outr localiz e NE,4
86,Não preeenchidos,603
87,Outros CID's não listados,34


In [3]:
data = data[data['CID Cancer'] != "Total"]
data

,CID Cancer,Notificações
0,C00 - Neopl malig do labio,3
1,C01 - Neopl malig da base da lingua,8
2,C02 - Neopl malig outr partes e NE da lingua,7
3,C04 - Neopl malig do assoalho da boca,4
4,C05 - Neopl malig do palato,3
...,...,...
83,D43 - Neopl comp inc/desc encefalo sist nerv c...,4
84,D46 - Sindr mielodisplasicas,6
85,D48 - Neopl comp inc/desconh outr localiz e NE,4
86,Não preeenchidos,603


In [4]:
dataForAnomalyDetection = data.drop(['CID Cancer'], axis=1)
dataForAnomalyDetection

,Notificações
0,3
1,8
2,7
3,4
4,3
...,...
83,4
84,6
85,4
86,603


In [5]:
from pycaret.anomaly import *

In [6]:
s = setup(dataForAnomalyDetection, session_id=123, system_log=False)

,Description,Value
0,Session id,123
1,Original data shape,"(88, 1)"
2,Transformed data shape,"(88, 1)"
3,Numeric features,1
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


In [7]:
models()

,Name,Reference
ID,,
abod,Angle-base Outlier Detection,pyod.models.abod.ABOD
cluster,Clustering-Based Local Outlier,pycaret.internal.patches.pyod.CBLOFForceToDouble
cof,Connectivity-Based Local Outlier,pyod.models.cof.COF
iforest,Isolation Forest,pyod.models.iforest.IForest
histogram,Histogram-based Outlier Detection,pyod.models.hbos.HBOS
knn,K-Nearest Neighbors Detector,pyod.models.knn.KNN
lof,Local Outlier Factor,pyod.models.lof.LOF
svm,One-class SVM detector,pyod.models.ocsvm.OCSVM
pca,Principal Component Analysis,pyod.models.pca.PCA


In [8]:
iforest = create_model('iforest')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,22:00:11
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


In [9]:
iforest

IForest(behaviour='new', bootstrap=False, contamination=0.05,
    max_features=1.0, max_samples='auto', n_estimators=100, n_jobs=-1,
    random_state=123, verbose=0)

In [10]:
plot_model(iforest, plot='umap', label=True)

In [11]:
result = assign_model(iforest)
result

,Notificações,Anomaly,Anomaly_Score
0,3,0,-0.226894
1,8,0,-0.186863
2,7,0,-0.211619
3,4,0,-0.234642
4,3,0,-0.226894
...,...,...,...
83,4,0,-0.234642
84,6,0,-0.227959
85,4,0,-0.234642
86,603,1,0.215896


In [12]:
cidCancerColumn = data['CID Cancer']

In [13]:
result.insert(0, 'CID Cancer', cidCancerColumn)
result

,CID Cancer,Notificações,Anomaly,Anomaly_Score
0,C00 - Neopl malig do labio,3,0,-0.226894
1,C01 - Neopl malig da base da lingua,8,0,-0.186863
2,C02 - Neopl malig outr partes e NE da lingua,7,0,-0.211619
3,C04 - Neopl malig do assoalho da boca,4,0,-0.234642
4,C05 - Neopl malig do palato,3,0,-0.226894
...,...,...,...,...
83,D43 - Neopl comp inc/desc encefalo sist nerv c...,4,0,-0.234642
84,D46 - Sindr mielodisplasicas,6,0,-0.227959
85,D48 - Neopl comp inc/desconh outr localiz e NE,4,0,-0.234642
86,Não preeenchidos,603,1,0.215896


In [14]:
result[result['Anomaly']==1]

,CID Cancer,Notificações,Anomaly,Anomaly_Score
27,C34 - Neopl malig dos bronquios e dos pulmoes,248,1,0.149788
34,C44 - Outr neopl malig da pele,1261,1,0.287152
39,C50 - Neopl malig da mama,144,1,0.033801
45,C61 - Neopl malig da prostata,134,1,0.007476
86,Não preeenchidos,603,1,0.215896


Fonte: Ministério da Saúde/SVS - Sistema de Informação de Agravos de Notificação - Sinan Net
Notas:

1. Períodos Disponíveis / Período - Correspondem aos anos de notificação dos casos.
1. CAT - Comunicação de acidente de trabalho O campo "CNAE - Ativ. Econ." considera os códigos referentes as tabelas CNAE 1.0.
1. Dados a partir 2020 do Espírito Santo não estão disponíveis neste sistema, pois são oriundos do Sistema de Informação e-SUS VS,
1. Dados de 2006 a 2017 revisados e atualizados em 15/08/2022.
1. Dados de 2018 revisados e atualizados em 22/08/2022.
1. Dados de 2019 atualizados em junho de 2023.
1. Dados de 2020 atualizados em junho de 2023, sujeitos à revisão.
1. Dados de 2021 revisados e atualizados em agosto de 2023, sujeitos à revisão.
1. Dados de 2022 revisados e atualizados em julho de 2023, sujeitos à revisão.
1. Dados de 2023 atualizados em janeiro de 2024, sujeitos à revisão.
* Dados disponibilizados no TABNET em fevereiro de 2024.